# Genetic Algorithm

## Hyperparams Optimization

### Setup
Change directory to the root folder to be able to import modules.

In [ ]:
import os

THIS_FOLDER = os.path.dirname(os.path.realpath("__file__"))
ROOT_FOLDER = os.path.dirname(THIS_FOLDER)
os.chdir(ROOT_FOLDER)

### Load Players Data

In [ ]:
from tests.helper import load_players

players = load_players()
print(f"There are {len(players)} players in this set.")

### Metric
Hyperparam needs a metric that reflects the goal.
The goal is to have the algorithm running enough time to find the best result, but not longer than the minimum necessary.

In [ ]:
import numpy as np
import seaborn as sns
import pandas as pd


def metric(points, time_elapsed):
    """Optimization metric."""
    return np.square(points) / np.sqrt(time_elapsed) / 1000


points_series = pd.Series(np.linspace(start=10, stop=100, num=10), name="Points")
time_series = pd.Series(np.linspace(start=1, stop=10, num=10), name="Time Elapsed")
df = pd.merge(points_series, time_series, how="cross")
df["Metric"] = metric(df["Points"], df["Time Elapsed"])
sns.heatmap(df.pivot(index="Points", columns="Time Elapsed", values="Metric"), cmap="Oranges");


### Tuning

In [ ]:
import time

import optuna
import optuna.logging

from cartola_draft import Scheme
from cartola_draft.algorithm.genetic import Genetic
from tests.helper import SCHEMES_COUNTING

optuna.logging.set_verbosity(optuna.logging.ERROR)

BUDGET = 100
SCHEME = Scheme(SCHEMES_COUNTING[442])


def algo_factory(
    n_generations,
    n_individuals,
    tournament_size_ratio,
    n_tournament_winners_ratio,
    max_n_mutations,
):
    """Create genetic algorithm insance."""

    tournament_size = tournament_size_ratio * n_individuals
    tournament_size = int(np.clip(round(tournament_size, 0), 2, None))

    n_tournament_winners = n_tournament_winners_ratio * tournament_size
    n_tournament_winners = int(np.clip(round(n_tournament_winners, 0), 1, None))

    return Genetic(
        players=players,
        n_generations=n_generations,
        n_individuals=n_individuals,
        tournament_size=tournament_size_ratio,
        n_tournament_winners=n_tournament_winners_ratio,
        max_n_mutations=max_n_mutations,
    )


def objective(trial):
    """Function to be optimized by optuna."""

    n_generations = trial.suggest_int("n_generations", 100, 500)
    n_individuals = trial.suggest_int("n_individuals", 100, 1000)
    tournament_size_ratio = trial.suggest_float("tournament_size_ratio", 0.1, 1)
    n_tournament_winners_ratio = trial.suggest_float("n_tournament_winners_ratio", 0.1, 1)
    max_n_mutations = trial.suggest_int("max_n_mutations", 1, 12)

    algo = algo_factory(
        n_generations,
        n_individuals,
        tournament_size_ratio,
        n_tournament_winners_ratio,
        max_n_mutations,
    )

    n_times = 5

    # Do it many times to avoid that variance bias the final decision.
    start = time.time()
    results = [algo.draft(BUDGET, SCHEME).points for _ in range(n_times)]
    end = time.time()

    # Discard largest and smallest value.
    results = sorted(results)[1:-1]
    points = np.mean(results)

    time_elapsed = (end - start) / n_times

    return metric(points, time_elapsed)


study = optuna.create_study()
study.optimize(objective, n_trials=100)

study.best_params


In [ ]:
algo = algo_factory(
        study.best_params["n_generations"],
        study.best_params["n_individuals"],
        study.best_params["tournament_size_ratio"],
        study.best_params["n_tournament_winners_ratio"],
        study.best_params["max_n_mutations"],
    )
print(f"Points = {algo.draft(BUDGET, SCHEME).points}")
print(f"Price = {algo.draft(BUDGET, SCHEME).price}")

### Analyze Results

In [ ]:
import optuna.visualization

fig = optuna.visualization.plot_optimization_history(study)
fig.show()

In [ ]:
fig = optuna.visualization.plot_param_importances(study)
fig.show()